In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments
import transformers
from datasets import Dataset, DatasetDict
from trl import SFTTrainer
import yaml
import random
import argparse

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def main(args):
    # set seed
    set_seed(0)

    # Model, Tokenizer
    model_name = "mistralai/Mistral-7B-Instruct-v0.1"
    access_token = ""

    model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=access_token)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Data
    json_file_path = os.path.join(args.dataset_file)

    # Load the JSON file directly into the train dataset
    train_dataset = Dataset.from_json(json_file_path)

    # Create the DatasetDict with the loaded train dataset
    ft_dict = DatasetDict({"train": train_dataset})

    num_iters=3
    # Training Arguments
    output_dir = args.output_path
    per_device_train_batch_size = 8
    num_train_epochs = 10
    gradient_accumulation_steps = 4
    optim = "paged_adamw_32bit"
    logging_strategy = "epoch"
    learning_rate = 9.46183526073646e-07
    max_grad_norm = 0.3
    warmup_ratio = 0.03
    lr_scheduler_type = "constant"

    training_arguments = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim=optim,
        logging_strategy=logging_strategy,
        learning_rate=learning_rate,
        group_by_length=True,
        max_grad_norm=max_grad_norm,
        warmup_ratio=warmup_ratio,
        lr_scheduler_type=lr_scheduler_type,
    )

    # Initializing Trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=ft_dict['train'],
        max_seq_length=1024,
        tokenizer=tokenizer,
        args=training_arguments
    )
    for i in range(num_iters):
        trainer.train()
        trainer.save_model(f"{output_dir}/checkpoint-{(i+1)*num_train_epochs}")
    # Saving the model after 10 additional epochs
    

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Fine-tuning script')
    parser.add_argument('--output_path', type=str, required=True, help='Path to save the fine-tuned model')
    parser.add_argument('--project_name', type=str, required=True, help='Name of the project')
    parser.add_argument('--dataset_file', type=str, required=True, help='Name of the dataset_path')
    args = parser.parse_args()

    # Call the main function with the parsed arguments
    main(args)